In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import seaborn as sns

Clean the table

In [3]:
def clean_table(db_path, table_name, id_columns=None, replace=True):
    """Clean table by removing duplicates and basic data cleaning"""
    # Connect to database
    conn = sqlite3.connect(db_path)
    print(f"Connected to {db_path}")
    
    # Load table to DataFrame
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    print(f"Original rows: {len(df)}")
    
    # Use all columns for deduplication if none specified
    if id_columns is None:
        id_columns = df.columns.tolist()
    
    # Remove duplicates
    df_clean = df.drop_duplicates(subset=id_columns)
    print(f"Removed {len(df) - len(df_clean)} duplicates")
    
    # Basic cleaning: handle nulls in string columns and trim whitespace
    for col in df_clean.select_dtypes(include=['object']):
        df_clean[col] = df_clean[col].fillna('').astype(str).str.strip()
    
    # Save cleaned data
    if replace:
        df_clean.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Replaced {table_name} with {len(df_clean)} clean rows")
    else:
        new_table = f"{table_name}_clean"
        df_clean.to_sql(new_table, conn, if_exists='replace', index=False)
        print(f"Created {new_table} with {len(df_clean)} clean rows")
    
    conn.close()
    return df_clean

In [4]:
# Example usage:
clean_df = clean_table("occupation_salaries (2).db", "Final_Combined_Data", replace=False)

Connected to occupation_salaries (2).db


In [2]:
# # Connect to your database - update the path to your actual database
# conn = sqlite3.connect("occupation_salaries (2).db")

# # Query to get your data
# query = '''SELECT YEAR, OCC_TITLE, AREA_TITLE, PRIM_STATE
#            FROM Final_Combined_Data_clean'''

# df = pd.read_sql_query(query, conn)
# conn.close()

# Connect to your database
conn = sqlite3.connect("occupation_salaries (2).db")

# Expanded query to get more columns
query = '''SELECT YEAR, OCC_TITLE, AREA_TITLE, PRIM_STATE, 
           tot_emp, jobs_1000, loc_quotient, h_mean, a_mean, h_median, a_median,
           emp_prse, mean_prse, naics, naics_title, i_group, own_code, occ_code,
           o_group, area_type
           FROM Final_Combined_Data_clean'''

df = pd.read_sql_query(query, conn)
conn.close()

# Print shape to confirm data was loaded
print(f"Loaded data with shape: {df.shape}")

Loaded data with shape: (4162883, 20)


Explore data

In [3]:
# Basic data exploration
print("Data shape:", df.shape)
print("\nSample data:")
print(df.head())
print("\nNull values:", df.isnull().sum())
print("\nUnique occupation titles:", df.OCC_TITLE.nunique())
print("\nYear range:", df.YEAR.min(), "to", df.YEAR.max())
# Print all columns in the DataFrame
print("DataFrame columns:")
print(list(df.columns))

Data shape: (4162883, 20)

Sample data:
   YEAR                        OCC_TITLE AREA_TITLE PRIM_STATE  TOT_EMP  \
0  2014                  All Occupations    Alabama             1857530   
1  2014           Management Occupations    Alabama               67500   
2  2014                 Chief Executives    Alabama                1080   
3  2014  General and Operations Managers    Alabama               26480   
4  2014                      Legislators    Alabama                1470   

  JOBS_1000 LOC_QUOTIENT H_MEAN    A_MEAN H_MEDIAN  A_MEDIAN EMP_PRSE  \
0    1000.0          1.0  19.66   40890.0    14.83   30850.0      0.4   
1    36.338         0.73  51.48  107080.0    44.98   93550.0      1.1   
2      0.58         0.32  97.67  203150.0        #         #      4.8   
3    14.258         0.94   58.0  120640.0     49.0  101930.0      1.5   
4      0.79         1.94      *   21920.0        *   18450.0      8.7   

  MEAN_PRSE   NAICS     NAICS_TITLE I_GROUP OWN_CODE OCC_CODE O_GROUP 

Encode data

In [4]:
# Encode categorical variables
le_occupation = LabelEncoder()
le_area = LabelEncoder()
le_state = LabelEncoder()

df['OCC_TITLE_ENCODED'] = le_occupation.fit_transform(df['OCC_TITLE'])
df['AREA_TITLE_ENCODED'] = le_area.fit_transform(df['AREA_TITLE'])
df['PRIM_STATE_ENCODED'] = le_state.fit_transform(df['PRIM_STATE'])

Feature engineering

In [5]:
# Create feature matrix with all available features
features = [col for col in df.columns if col not in ['YEAR', 'TOT_EMP', 'JOBS_1000', 
            'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN', 'EMP_PRSE', 
            'MEAN_PRSE', 'AREA_TYPE', 'OCC_TITLE_ENCODED', 'AREA_TITLE_ENCODED', 'PRIM_STATE_ENCODED']]
X = df[features].copy()

# Handle missing values
for col in X.columns:
    if X[col].dtype.kind in 'fiuO':  # numeric or object columns
        X[col] = X[col].fillna(0 if X[col].dtype.kind in 'fiu' else 'unknown')

y = df['OCC_TITLE_ENCODED']

In [6]:
# Convert object columns to numeric if they contain numbers
object_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"Converting {len(object_cols)} object columns to numeric")

for col in object_cols:
    # Try to convert to numeric, errors='coerce' will set non-convertible values to NaN
    X[col] = pd.to_numeric(X[col], errors='coerce')
    
    # Fill NaN values with 0
    X[col] = X[col].fillna(0)
    
# Check the data types after conversion
print("\nData types after numeric conversion:")
print(X.dtypes)

# Verify we have no object columns left
remaining_objects = X.select_dtypes(include=['object']).columns.tolist()
if remaining_objects:
    print(f"\nStill have {len(remaining_objects)} object columns: {remaining_objects}")
else:
    print("\nAll columns successfully converted to numeric types")

Converting 9 object columns to numeric

Data types after numeric conversion:
OCC_TITLE      float64
AREA_TITLE     float64
PRIM_STATE     float64
NAICS          float64
NAICS_TITLE    float64
I_GROUP        float64
OWN_CODE       float64
OCC_CODE       float64
O_GROUP        float64
dtype: object

All columns successfully converted to numeric types


Split data by YEAR - train: all be last year, test: last year

In [9]:
# Split data by time
years = df.YEAR.unique()
years.sort()
train_years = years[:-1]  # Use all but the last year for training
test_year = years[-1]     # Use the last year for testing

X_train = X[df.YEAR.isin(train_years)]
y_train = y[df.YEAR.isin(train_years)]
X_test = X[df.YEAR == test_year]
y_test = y[df.YEAR == test_year]

print(f"\nTraining on years: {train_years}")
print(f"Testing on year: {test_year}")
print(f"Total training samples: {len(X_train)}")


Training on years: ['2014' '2015' '2016' '2017' '2018' '2019' '2020' '2021' '2022']
Testing on year: 2023
Total training samples: 3749556


In [10]:
# Sample the training data to speed up feature selection
sample_size = 0.05  # Use 5% of the data
sample_indices = np.random.choice(len(X_train), size=int(len(X_train) * sample_size), replace=False)
X_train_sample = X_train.iloc[sample_indices]
y_train_sample = y_train.iloc[sample_indices]

print(f"Using {len(X_train_sample)} samples ({sample_size*100:.1f}%) for feature selection")


Using 187477 samples (5.0%) for feature selection


In [11]:
from sklearn.feature_selection import SelectFromModel

# Use a model for feature selection
selector = SelectFromModel(RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42))
selector.fit(X_train_sample, y_train_sample)

# Get selected features
feature_mask = selector.get_support()
selected_features = X.columns[feature_mask]
print(f"\nSelected {len(selected_features)} features out of {len(X.columns)}:")
print(selected_features.tolist())

# Create datasets with only selected features
X_train_selected = X_train.loc[:, feature_mask]
X_test_selected = X_test.loc[:, feature_mask]

print(f"\nOriginal feature count: {X_train.shape[1]}")
print(f"Reduced feature count: {X_train_selected.shape[1]}")

# Print top 10 feature importance scores
if hasattr(selector.estimator_, 'feature_importances_'):
    importances = selector.estimator_.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    print("\nTop 10 features by importance:")
    for i, idx in enumerate(indices[:10]):
        print(f"{i+1}. {X.columns[idx]} ({importances[idx]:.4f})")


Selected 2 features out of 9:
['NAICS', 'OWN_CODE']

Original feature count: 9
Reduced feature count: 2

Top 10 features by importance:
1. NAICS (0.8732)
2. OWN_CODE (0.1268)
3. O_GROUP (0.0000)
4. OCC_CODE (0.0000)
5. I_GROUP (0.0000)
6. NAICS_TITLE (0.0000)
7. PRIM_STATE (0.0000)
8. AREA_TITLE (0.0000)
9. OCC_TITLE (0.0000)


Using all features to train/test model

In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet

# Set quieter logging for Prophet and smaller-sized models
import logging
logging.getLogger('prophet').setLevel(logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

/Applications/OMSA Course Work/CSE 4242 - DVA/Final Project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [8]:
# Directly select these specific columns as features - not exclude them
feature_columns = ['YEAR', 'TOT_EMP', 'JOBS_1000', 
                  'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN', 'EMP_PRSE', 
                  'MEAN_PRSE', 'AREA_TYPE', 'OCC_TITLE_ENCODED', 'AREA_TITLE_ENCODED', 'PRIM_STATE_ENCODED']

# Define X and y - explicitly use the columns you specified
X = df[feature_columns].copy()
y = df['OCC_TITLE_ENCODED']

# Convert object columns to numeric
object_cols = X.select_dtypes(include=['object']).columns.tolist()
for col in object_cols:
    X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)

# Split data by time
years = df.YEAR.unique()
years.sort()
train_years = years[:-1]  # Use all but the last year for training
test_year = years[-1]     # Use the last year for testing

# Split the data
X_train = X[df.YEAR.isin(train_years)]
y_train = y[df.YEAR.isin(train_years)]
X_test = X[df.YEAR == test_year]
y_test = y[df.YEAR == test_year]

print(f"Training on years: {train_years}, Testing on year: {test_year}")
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

# Sample training data for faster processing
sample_size = 0.05  # Use 5% of the data
np.random.seed(42)
sample_indices = np.random.choice(len(X_train), size=int(len(X_train) * sample_size), replace=False)
X_train_sample = X_train.iloc[sample_indices]
y_train_sample = y_train.iloc[sample_indices]

print(f"Using {len(X_train_sample)} samples ({sample_size*100:.1f}%) for model training")


Training on years: ['2014' '2015' '2016' '2017' '2018' '2019' '2020' '2021' '2022'], Testing on year: 2023
Training samples: 3749556, Testing samples: 413327
Using 187477 samples (5.0%) for model training


Random Forests Model

In [ ]:
# Train Random Forest model
print("Training Random Forest model...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_sample, y_train_sample)

# Make predictions
rf_predictions = rf_model.predict(X_test)

# Evaluate
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy on test year {test_year}: {rf_accuracy:.4f}")

# Print feature importance
feature_importances = pd.DataFrame({
    'Feature': X_train_sample.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance:")
print(feature_importances)

# Save the model for later use
import joblib
joblib.dump(rf_model, 'random_forest_model.joblib')
print("Model saved as 'random_forest_model.joblib'")